In [7]:
import requests
import random
import json

base_url = "http://localhost:6083"

resp = requests.get(base_url)
assert resp.status_code == 404
assert resp.json()["status"] == 404
assert resp.json()["error"] == "Not Found"


In [8]:

resp = requests.get("%s/api/docs/swagger.json"%base_url)
assert resp.status_code == 200
assert resp.json()["swagger"] == "2.0"
assert resp.json()["paths"] != None

swaggerJson = resp.json()
swaggerJson["paths"].keys()

jsonUrls = [f for f in swaggerJson["paths"].keys() if ".json" in f]
print(jsonUrls)
jsonUrl = jsonUrls[0]

resp = requests.get("%s%s"%(base_url, jsonUrl))
assert resp.status_code == 200


['/api/docs/Issuer.json', '/api/docs/Schema.json', '/api/docs/DocumentType.json']


In [9]:
resp.json()

{'Issuer': {'$id': '#/properties/Issuer',
  'type': 'object',
  'title': 'Issuer',
  'required': [],
  'properties': {'name': {'type': 'string',
    'title': 'Issuer/Organisation Name'},
   'sectorType': {'type': 'string',
    'enum': ['Education'],
    'title': 'Sector Type'},
   'logoUrl': {'type': 'string', 'title': 'Logo url'},
   'websiteUrl': {'type': 'string', 'title': 'Website url'},
   'accountDetails': {'description': 'Enter Email id or Phone number that you would use for login',
    'title': 'Account Details',
    'type': 'object',
    'properties': {'userId': {'type': 'string',
      'title': 'Email Id or Mobile number'}}},
   'contactDetails': {'type': 'object',
    'description': 'Enter administrator details',
    'title': 'Contact Details',
    'properties': {'name': {'type': 'string', 'title': 'Contact Name'},
     'mobile': {'type': 'string', 'title': 'Mobile number'},
     'email': {'type': 'string', 'title': 'Email ID'}}},
   'schemas': {'type': 'array',
    'items':

In [10]:
entities = list(resp.json().keys())
print("Available entities ", entities)

Available entities  ['Issuer']


In [5]:
entity_name=entities[0]
print("Using entity %s"%entity_name)


Using entity TrainingCertificate


In [6]:
userId =str(random.randint(1e10,1e11))
resp = requests.post("%s%s"%(base_url, '/api/v1/%s'%entity_name), json={
    "name":"Sunbird Learner", 
    "contact": userId, 
    "trainingTitle":"Sunbird RC Certificate Module"
    
})
assert resp.status_code == 200 or print (resp.json())
idx = resp.json()["result"][entity_name]["osid"]


{'id': 'sunbird-rc.registry.create', 'ver': '1.0', 'ets': 1647412648849, 'params': {'resmsgid': '', 'msgid': 'cf15acc7-1c30-43da-930b-55b61ffb482a', 'err': '', 'status': 'UNSUCCESSFUL', 'errmsg': 'java.lang.RuntimeException: java.sql.SQLClientInfoException: Failed to set ClientInfo property: ApplicationName'}, 'responseCode': 'OK', 'result': {}}


AssertionError: 

In [54]:
resp.json()


{'id': 'sunbird-rc.registry.create',
 'ver': '1.0',
 'ets': 1646379652331,
 'params': {'resmsgid': '',
  'msgid': 'f1d009a3-79c8-4172-ad4e-7d043a978874',
  'err': '',
  'status': 'SUCCESSFUL',
  'errmsg': ''},
 'responseCode': 'OK',
 'result': {'TrainingCertificate': {'osid': '1-121ee8b7-62e1-42a4-8f70-94d558161c6c'}}}

In [55]:
print(resp.json())
resp.status_code, userId


{'id': 'sunbird-rc.registry.create', 'ver': '1.0', 'ets': 1646379652331, 'params': {'resmsgid': '', 'msgid': 'f1d009a3-79c8-4172-ad4e-7d043a978874', 'err': '', 'status': 'SUCCESSFUL', 'errmsg': ''}, 'responseCode': 'OK', 'result': {'TrainingCertificate': {'osid': '1-121ee8b7-62e1-42a4-8f70-94d558161c6c'}}}


(200, '91912894296')

In [56]:
resp = requests.get("%s/api/v1/%s/%s"%(base_url, entity_name, idx), headers={"Accept":"application/vc+ld+json"})
print(json.dumps(resp.json()))
assert resp.json()["proof"]["type"] == "Ed25519Signature2018"


{"@context": ["https://www.w3.org/2018/credentials/v1", "https://gist.githubusercontent.com/dileepbapat/eb932596a70f75016411cc871113a789/raw/498e5af1d94784f114b32c1ab827f951a8a24def/skill"], "type": ["VerifiableCredential"], "issuanceDate": "2021-08-27T10:57:57.237Z", "credentialSubject": {"type": "Person", "name": "Sunbird Learner", "trainedOn": "Sunbird RC Certificate Module"}, "issuer": "did:web:sunbirdrc.dev/vc/skill", "proof": {"type": "Ed25519Signature2018", "created": "2022-03-04T07:40:52Z", "verificationMethod": "did:india", "proofPurpose": "assertionMethod", "jws": "eyJhbGciOiJFZERTQSIsImI2NCI6ZmFsc2UsImNyaXQiOlsiYjY0Il19..EFn8QZk9znnADcQNlir-Wv1HTlKyJMUSe07pxoct-J3i7-6N5Uf5P26d6fu0_TvciOFDVyChFAa35CoO9bbsAA"}}


In [60]:
resp = requests.get("%s/api/v1/%s/%s"%(base_url, entity_name, idx), headers={"Accept":"application/pdf", "template": "https://raw.githubusercontent.com/dileepbapat/ref-sunbirdrc-certificate/main/schemas/templates/TrainingCertificate.html"})


In [63]:
(resp.status_code, resp.content)

assert resp.content[:5].decode().startswith("%PDF")
with open('sample.pdf', 'wb') as f:
    f.write(resp.content)

%system open 'sample.pdf'

[]

In [72]:
resp = requests.get("%s/api/v1/%s/%s"%(base_url, entity_name, idx), headers={"Accept":"application/pdf", "template-key": "html"})
(resp.status_code, resp.content)

assert resp.content[:5].decode().startswith("%PDF")
with open('sample.pdf', 'wb') as f:
    f.write(resp.content)
    %system open 'sample.pdf'

In [70]:
%system open 'sample.pdf'

[]

In [13]:
resp = requests.get("%s/api/v1/%s/%s"%(base_url, entity_name, idx), headers={"Accept":"text/html"})
resp.status_code, resp.content

assert resp.content[:5].decode().startswith("<html")
with open('sample.html', 'wb') as f:
    f.write(resp.content)
%system open 'sample.html'


[]

In [14]:
resp = requests.get("%s/api/v1/%s/%s"%(base_url, entity_name, idx), headers={"Accept":"image/svg+xml"})
resp.status_code, resp.content

assert resp.content.decode().find("<svg") != -1
with open('sample.svg', 'wb') as f:
    f.write(resp.content)
%system open 'sample.svg'

[]